## Comparison with Profound and pybdsf.

find a image/field with a lot of sources that has a pybdsf catalogue (e.g. COSMO & XMM-LSS Fields.)

If there is not one, create a ProFound catalogue for a field and compare it with pybdsf.

Createa a TRSF field and compare with the others.

This to compare, the number of sources, the fluxes, the positions, the sizes, etc.

How extended sources are treated. Test on some complex AGN sources, 3C Catalogue, both pybdsf and Profound.




### Source finding (Haywood et al 2021)
We used the pybdsf source finder (Mohan & Rafferty 2015) to locate and characterise components in the primary beam corrected COSMOS image, and the XMM-LSS mosaic. Briefly, pybdsf works by using a sliding box to estimate the local background noise (σlocal) as a function of position in the image. It then locates pixels in the image whose brightness exceeds the local background noise by some factor (in this case 5σlocal). A flood-fill algorithm is then used to identify islands of contiguous emission down to some secondary threshold, in this case 3σlocal. These islands are then iteratively fitted with point and 2D Gaussian components, and pybdsf then attempts to group point and Gaussian components into sources, based on the brightness of the pixels between components in relation to the secondary threshold, and a separation criterion based on the measured sizes of the components. A catalogue describing the properties of the components and source groupings is then exported, and this raw output forms our Level-0 catalogue products. The Level-0 catalogues derived from the robust 0.0 images contain 9,915 and 20,397 components in COSMOS and XMM-LSS respectively. The steps in the sections that follow describe the modifications and addition.

Note since we are comparing default outputs of these functions, we do not want to look at the level 1 catalogoues as these have been altered through visul insepection.

# Code used to replicate the source catalogoue creation process of Hale C, et al 2019.

Blind Detection:
Run a blind detection with PROFOUND using profoundProFound. Run this over the image using skycut = 3.5 and set groupstats = TRUE and groupby = ‘segim’.Each source is defined as a segment.
```R
image = readFITS(image_file)
image_blind=profoundProFound(image,plot=False, skycut=3.5, rotstats=TRUE,boundstats=TRUE,nearstats=TRUE,groupstats=TRUE, groupby='sgim', verbose=TRUE)
write.csv(image_blind$groupstats,file='file_name.csv', quote=FALSE,row.names=FALSE)
```
Using the grouped segments (group$groupim) and the corresponding statistics (properties) for these segments from groupstats, a catalogue of sources can be defined. Using the grouped segments ensures that any adjacent segments are combined and the source information for the merged segment is recorded within the source catalogue.

```R
segim_model =image_blind$group$groupim
sgim_model[image_blind$blind$group$groupim!=0] = 1
segim_model[image_blind$group$groupim==0] = as.numeric(NaN)
model = (image$imDat - image_blind$sky)
```

Apply a beam correction to convert between the map (in Jy/beam) to the total flux densities (in Jy).
This ensures that neighbouring segments are combined together.



### PROFOUND Catalogue Headings Meaning
| Column Name | Description |
|-------------|-------------|
|segID: |Segmentation ID, which can be mathced against values in 'segim'
|uniqueID: |statics and based on the xmax and ymax positions.
|xcen: |flux weighted x centre
|ycen: |flux weighted y centre
|xmax: |x position of maximum flux
|ymax: |y position of maximum flux
|RAcen: |Flux weighted Right ascension centre in degrees. (requires header)
|Deccen: |Flux weighted Delination centre (requires header)
|RAmax: |Right acsension of max flux (requires header)
|Decmax: |Declination of max flux (required header)
sep: |radial offset between the cen and max definition of the centre (in pixel units)
flux: |Total flux
mag: |Total flux converted to mag using 'mag zero'
flux_app: |Psuedo aperture flux
mag_app: |Psuedo aperture flux converted to mag using 'magzero'
N50: | number of brightest pixels contatining 50% of the flux
N90: |number of brightest pixels containing 90% of the flux
N100: |Total number of pixels in this segement, i.e contains 100% of the flux.
R50: |approximate eeliptical semi-major axis containing 50% of the flux (pixel units)
R90: |approxiamte elipitical semi-major axis containing 90% of the flux (pixel units)
R100: |approxamate elliptical semi-major axis containing 100% of the flux.
SB_N50: |Mean surface brightness containing bightest 50% of the flux Calculated using (flux*0.5/N50)
SB_N90: |Mean surface brightness containing bightest 90% of the flux Calculated using (flux*0.5/N90)
SB_N100: |Mean surface brightness containing bightest 100% of the flux Calculated using (flux*0.5/N100)
xsd: |weighted standard deviasion in x (pixel units)
ysd: |weighted standard deviation in y (pixel units)
covxy: |weighted covarience in xy (pixel units)
corxy: |weighted correlation in xy (pixel units)
con: |concentration (R50/R90)
asymm: |180 degree flux asymmetry (0-1, where o os perfect symmetry and 1 is complete asymmetry)
flux_reflect: |Flux corrected asymmetry by doubling the contribution of flux for asymetric pixels.
mag_reflectL: |'flux_reflect' converted to mag using 'magzero'
semimaj: |weighted standard deviation along the major axis, i.e. the semi-major first moment, so -2 times this would be a typical axis kron radius (pixel units).
semimin: |weighted standard deviation along the minor axis, i.e. the semi-minor first moment, so -2 times this would be a typical axis kron radius (pixel units).
axrat: |axial ratio as given by min/maj
ang: |Orientation oif the semi-major axis in degress.
signif: |Approximate singificance  of the detectio usin gchi-squared method.
FPlim: |Approxiamte false-positive seignificance limit below which one such source might appear spuriously on an image.
flux_err: |estimated total error in the flux for the seggment
mag_err: |flux erro rin magnitude
flux_err_sky: |Sky subtraction component of the flux error
flux_err_skyRMS: |Sky RMS component of the flux error
flux_err_shot: |Object shot-noise componet of the flux error
flux_err_cor: |Error component due to pixel correlation
cor_seg: |??
sky_mean: |Mean flux of the sky over all segemenbt pixels.
sky_sum: |Total flux fo the sky over all segment pixels
skyRMS_mean: |Mean value of the sky RMS over all segment pixels
Nedge:| Number of edge segmen pixels that make up the outer edge of the segment
Nsky: |Number of edge pizels that are touching the sky.
Nobject:| Number of edge segement pixels that are touching anouther object segment.
Nborder:| Number of edge pixels that are touching the 'image' border.
Nmask:| Number of edge segmen pixels hthat are touching a masked pixel.
edge_fra: |fraction of edge segement pixels that are touching the sky.
edge_excess: |Ratio of the number of edge pixels to the expected number given the elliptical geometry measurements of the segmen. If this is larger than 1 then it is a s sign that the segment geometry is irregualr, and is likely a flag for compromised photometry.
flag_border:| A binary flag telling the user which 'image' boarder the segement touches. The bottom of the 'imageo/ is flagged 1, left=2,top=4,and right =8. A summed combination of these flags indicate the segement o in a corner touching two borders: bottom-left=3,top-left=6,top-right=12, bottom-right=9.

### pyBDSF Catalogue headers
| Header | Description |
|--------|-------------|
Gaus_id| 
Isl_id| a unique number that identifies the Island, starting from zero
Source_id| a unique number that identifies the Source, starting from zero
Wave_id|
RA| the right ascension of the source (for the equinox of the image)
E_RA| the error on the right ascension of the source
DEC| the declination of the source (for the equinox of the image)
E_DEC| the 1-sigma error on the declination of the source
Total_flux| the total, integrated Stokes I flux density of the source at the reference frequency
E_Total_flux| the 1-sigma error on the total flux density of the source
Peak_flux| the peak Stokes I flux density per beam of the source
E_Peak_flux| the 1-sigma error on the peak flux density per beam of the source
Xposn| 
E_Xposn| 
Yposn|
E_Yposn| 
Maj| 	the FWHM of the major axis of the source
E_Maj| the 1-sigma error on the FWHM of the major axis of the source
Min|the FWHM of the minor axis of the source
E_Min| the 1-sigma error on the FWHM of the minor axis of the source
PA| the position angle of the major axis of the source measured east of north
E_PA| the 1-sigma error on the position angle of the major axis of the source
Maj_img_plane| the FWHM of the major axis of the source in the image plane
E_Maj_img_plane| the 1-sigma error on the FWHM of the major axis of the source in the image plane
Min_img_plane| the FWHM of the minor axis of the source in the image plane
E_Min_img_plane| the 1-sigma error on the FWHM of the minor axis of the source in the image plane
PA_img_plane| 	the position angle in the image plane of the major axis of the source measured east of north
E_PA_img_plane| the 1-sigma error on the position angle in the image plane of the major axis of the source
DC_Maj| the FWHM of the deconvolved major axis of the source
E_DC_Maj| the 1-sigma error on the FWHM of the deconvolved major axis of the source
DC_Min| The FWHM of the deconvolved minor axis of the source
E_DC_Min| The 1-sigma error on the FWHM of the deconvolved minor axis of the source
DC_PA| the position angle of the deconvolved major axis of the source measured east of north
E_DC_PA| the 1-sigma error on the position angle of the deconvolved major axis of the source
DC_Maj_img_plane| the FWHM of the deconvolved major axis of the source in the image plane
E_DC_Maj_img_plane| the 1-sigma error on the FWHM of the deconvolved major axis of the source in the image plane
DC_Min_img_plane| the FWHM of the deconvolved minor axis of the source in the image plane
E_DC_Min_img_plane| the 1-sigma error on the FWHM of the deconvolved minor axis of the source in the image plane
DC_PA_img_plane| the position angle in the image plane of the deconvolved major axis of the source measured east of north
E_DC_PA_img_plane| 	the 1-sigma error on the position angle in the image plane of the deconvolved major axis of the source
Isl_Total_flux| the total, integrated Stokes I flux density of the island in which the source is located. This value is calculated from the sum of all non-masked pixels in the island with values above thresh_isl
E_Isl_Total_flux| 	the 1-sigma error on the total flux density of the island in which the source is located
Isl_rms| the average background rms value of the island
Isl_mean| the averge background mean value of the island
Resid_Isl_rms| the average residual background rms value of the island
Resid_Isl_mean| the averge residual background mean value of the island
S_Code| A code that defines the source structure 'S' for single Gaussian, 'M' for multiple Gaussian, 'C' = a single-Gaussian source in an island with other sources

#  Data

MIGHTEE Observations of the COSMO and XMM-LSS Fields. 

These pyBDSF component catalogues have been published by Heywood,I. et al 2021.

Caviate to this data sources is that the primary beam calibrations could be incorrect?

Images:
1. Robust 0.0 - Resolution 8" (confusion limited images to roughly half this depth.) Thermal noise is below 2 uJy/Beam
2. Robust -1.2 - High Resolution (not confusion limited, but have thermal noise that is three times higher). 

Each of the above has a pybdsf Catalogue.


## COSMO Field



# XMM-LSS

# Tests


## Completeness and Reliabilty

Inject sources into the fields and measure how many are recoverd. These can be selected from a random distrubtion. How does this change with source peak brightness.


Compare its true flux density with the detected flux density from the three source finders.




What I want to compare, is how it can extract/deal with complex sources. that can be fitted with multiple features.
The purpose is that we can make a decision later about whether this detected components are part of a larger sources or not.

1. Apply source finders on a complex sources with other sources on it.

2. Create residuals/ models of the fitted sources.

3. Compare how the source finders have described the exteneded regions, how they distingusish between different areas etc.

# Create cutouts from SKA DC1 image for samples of these complex sources.


